# *Machine Learning Project Walkthrough: Data Cleaning*
### Reading in to Pandas

In [28]:
import pandas as pd

loans_2007 = pd.read_csv('loans_2007.csv')
print(len(loans_2007.columns))
loans_2007.head()

52


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### First Group of Columns

In [29]:
loans_2007.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 
                 'issue_d'], axis=1, inplace=True)
len(loans_2007.columns)

44

### Second Group of Columns

In [30]:
loans_2007.drop(['zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
                 'total_rec_prncp'], axis=1, inplace=True)
len(loans_2007.columns)

38

### Third Group of Columns

In [31]:
loans_2007.drop(['total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
                 'last_pymnt_d', 'last_pymnt_amnt'], axis=1, inplace=True)
# print(len(loans_2007.columns))
print(loans_2007.shape)

loans_2007.head()

(42538, 32)


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,other,Personal,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### Target Column

In [32]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

### Binary Classification

In [36]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]

loan_status_dict = {
    'loan_status': {
        'Fully Paid' : 1,
        'Charged Off' : 0
    }
}

loans_2007 = loans_2007.replace(loan_status_dict)

print(loans_2007.shape)
loans_2007.head()

(38770, 32)


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### Removing Single Value Columns

In [40]:
columns = loans_2007.columns
drop_columns = []

for item in columns:
    non_null = loans_2007[item].dropna()
    non_null_unique = non_null.unique()
    unique_val = len(non_null_unique)
    
    if unique_val == 1:
        drop_columns.append(item)
        
print(drop_columns)

loans_2007.drop(drop_columns, axis=1, inplace=True)
loans_2007.head()

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,Jun-2016,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,Sep-2013,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,Jun-2016,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,Apr-2016,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,Jan-2016,0.0


# *Machine Learning Project Walkthrough: Preparing the Features*
### Recap

In [59]:
loans = pd.read_csv('filtered_loans_2007.csv')
null_counts = loans.isnull().sum()
null_counts[null_counts > 0]

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

### Handling Missing Values

In [60]:
loans.drop('pub_rec_bankruptcies', axis=1, inplace=True)
loans.dropna(inplace=True)

loans.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

### Text Columns

In [61]:
object_columns_df = loans.select_dtypes(include=['object'])
object_columns_df.head(1)

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016


### First 5 Categorical columns

In [62]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for item in cols:
    print(item)
    print(loans[item].value_counts(), '\n')

home_ownership
RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64 

verification_status
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64 

emp_length
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64 

term
 36 months    28234
 60 months     9441
Name: term, dtype: int64 

addr_state
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162


### The Reason for the Loan

In [63]:
for item in ['purpose', 'title']:
    print(item)
    print(loans[item].value_counts(), '\n')

purpose
debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64 

title
Debt Consolidation                          2068
Debt Consolidation Loan                     1599
Personal Loan                                624
Consolidation                                488
debt consolidation                           466
Credit Card Consolidation                    345
Home Improvement                             336
Debt consolidation                           314
Small Business Loan                          298
Credit Card Loan                             294
Personal                                     290
Consolidation Loan       

### Categorical Columns

In [64]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

## Dropping Columns which require a lot of feature engineering or make a lot of dummy variables
loans.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1, inplace=True)

## Stripping columns of '%' sign and converting to float
for item in ['int_rate', 'revol_util']:
    loans[item] = loans[item].str.rstrip('%').astype(float)
    
## Replacing values in emp length from text to numerical years
loans.replace(mapping_dict, inplace=True)

loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
4,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


### Dummy Variables

In [65]:
dummy_cols = ['home_ownership', 'verification_status', 'purpose', 'term']

dummy_df = pd.get_dummies(loans[dummy_cols])
loans = pd.concat([loans, dummy_df], axis=1)
loans.drop(dummy_cols, axis=1, inplace=True)
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


# *Machine Learning Project Walkthrough: Preparing the Features*
### Recap

In [1]:
import pandas as pd
loans = pd.read_csv('cleaned_loans_2007.csv')
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38708 entries, 0 to 38707
Data columns (total 38 columns):
loan_amnt                              38708 non-null float64
int_rate                               38708 non-null float64
installment                            38708 non-null float64
emp_length                             38708 non-null int64
annual_inc                             38708 non-null float64
loan_status                            38708 non-null int64
dti                                    38708 non-null float64
delinq_2yrs                            38708 non-null float64
inq_last_6mths                         38708 non-null float64
open_acc                               38708 non-null float64
pub_rec                                38708 non-null float64
revol_bal                              38708 non-null float64
revol_util                             38708 non-null float64
total_acc                              38708 non-null float64
home_ownership_MORTGAGE    

### Picking an Error Metric

In [5]:
import numpy as np

# Predict that all loans will be paid off on time.
predictions = pd.Series(np.ones(loans.shape[0]))

tn_filter = (predictions == 0) &  (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])
print(tn)

tp_filter = (predictions == 1) &  (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])
print(tp)

fn_filter = (predictions == 0) &  (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])
print(fn)

fp_filter = (predictions == 1) &  (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])
print(fp)

0
33093
0
5615


### Class imbalance

In [6]:
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

1.0
1.0


### Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')

features = loans.drop('loan_status', axis=1)
target = loans['loan_status']

lr.fit(features, target)
predictions = lr.predict(features)

### Cross Validation

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(solver='lbfgs')

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

tn_filter = (predictions == 0) &  (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) &  (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) &  (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) &  (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])

## Printing False Positive and True Positive Rate
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

/home/naveen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.9966162065894925
0.9987006315535008


### Penalizing the Classifier

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(solver='lbfgs', class_weight='balanced')

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

tn_filter = (predictions == 0) &  (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) &  (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) &  (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) &  (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])

## Printing False Positive and True Positive Rate
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

0.3437221727515583
0.5331338953857311


### Manual Penalties

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

penalty = {
    0:10,
    1:1
}

lr = LogisticRegression(solver='lbfgs', class_weight=penalty)

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

tn_filter = (predictions == 0) &  (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) &  (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) &  (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) &  (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])

## Printing False Positive and True Positive Rate
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

/home/naveen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/naveen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.10739091718610863
0.21829389901187563


### Random Forests

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

rf  = RandomForestClassifier(class_weight='balanced', random_state = 1)

predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

tn_filter = (predictions == 0) &  (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) &  (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) &  (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) &  (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])

## Printing False Positive and True Positive Rate
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

/home/naveen/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/naveen/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/naveen/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9271593944790739
0.9709304082434351
